In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

first, import module what i needed

In [ ]:
import pandas as pd

read and check the csv files.

In [ ]:
corna_rok = pd.read_csv("/kaggle/input/covid19-in-south-korea/corona_rok.csv")
mers_rok = pd.read_csv("/kaggle/input/covid19-in-south-korea/mers_rok.csv")
Sars_rok = pd.read_csv("/kaggle/input/covid19-in-south-korea/2003-SARS_rok.csv")

- 이 데이터는 한국에서 발생 했었던 사스, 메르스, 코로나에 관한 것입니다. 
- 공통적으로 발생한 시기, 확진자, 사망자, 완치자가 기록되어 있습니다. 
- 첫 미니 프로젝트인 이유로 조금은 가볍게 접근해 보고자 하였습니다. 
- 과정은 다음과 같습니다.
- 1. 데이터를 분석 목적에 맞게 전처리 한다.
- 2. 최종 목표는 시계열 분석을 사용하여 확진자 수를 예측하는 것이고, 정확도를 비교하기 위해 이동평균을 사용해보고자 합니다.

In [ ]:
print(corna_rok)
print(corna_rok.info())

In [ ]:
corna_rok

In [ ]:
print(mers_rok.head())

In [ ]:
print(Sars_rok.head())
print(Sars_rok.info())

- Corna and Sars files are mainly consists of 'date', 'confrimed, and 'deaths'.
- but Mers file is consist of many columns that i don't needed.
- so i will remove useless columns and use 'groupby' function to calcuate counts based 'date' column in Mers file.

In [ ]:
mres_rok_pre = mers_rok[['reported','number','death']].groupby(['reported'],as_index = False).count()

In [ ]:
mres_rok_pre

- three dataset's date aren't same at all. so i use theirs index only to draw visualization

In [ ]:
corna_rok['index'] = pd.Series(['1','2','3','4','5','6','7','8','9','10','11','12','13','14','15','16','17','18','19','20','21','22','23','24','25','26','27','28','29','30','31','32','33','34','35','36','37','38','39','40','41','42','43','44','45','46','47','48','49','50','51','52','53','54','55','56','57','58','59','60','61','62','63','64','65'], index = corna_rok.index)
corna_rok

In [ ]:
Sars_rok['index'] = pd.Series(['1','2','3','4','5','6','7','8','9','10','11','12','13','14','15','16','17','18','19','20','21','22','23','24','25','26','27','28','29','30','31','32','33','34','35','36','37','38','39','40','41','42','43','44','45','46','47','48','49','50','51','52','53','54','55','56','57','58','59'], index = Sars_rok.index)
Sars_rok

In [ ]:
mres_rok_pre['index'] = pd.Series(['1','2','3','4','5','6','7','8','9','10','11','12','13','14','15','16','17','18','19','20','21','22','23','24','25','26','27','28','29','30','31','32','33','34','35'], index = mres_rok_pre.index)
mres_rok_pre

In [ ]:
# import module to draw plots

import numpy as np
import plotly.offline as pyo
import plotly.graph_objs as go

In [ ]:
trace1 = go.Scatter(x = corna_rok['index'], y = corna_rok['confirmed'], mode = 'lines', name = 'corna_rok')
trace2 = go.Scatter(x = Sars_rok['index'], y = Sars_rok['confirmed'], mode = 'lines', name = 'Sars_rok')
trace3 = go.Scatter(x = mres_rok_pre['index'], y = mres_rok_pre['number'], mode = 'lines', name = 'mres_rok_pre')

data = [trace1, trace2, trace3]
pyo.iplot(data)

- their values are too unbalnce to compare. therefore, i will use normalization all values.

In [ ]:
from sklearn.preprocessing import MinMaxScaler
print(corna_rok['confirmed'].tail())
print(Sars_rok['confirmed'].head())
print(mres_rok_pre['number'].head())

In [ ]:
scaler = MinMaxScaler()
corna_rok['confirmed_scaled'] = scaler.fit_transform(corna_rok['confirmed'].values.reshape(-1,1))
mres_rok_pre['confirmed_scaled'] = scaler.fit_transform(mres_rok_pre['number'].values.reshape(-1,1))
Sars_rok['confirmed_scaled'] = scaler.fit_transform(Sars_rok['confirmed'].values.reshape(-1,1))

In [ ]:
corna_rok.head()

In [ ]:
mres_rok_pre.head()

In [ ]:
Sars_rok.head()

In [ ]:
trace1 = go.Scatter(x = corna_rok['index'], y = corna_rok['confirmed_scaled'], mode = 'lines', name = 'corna_rok')
trace2 = go.Scatter(x = Sars_rok['index'], y = Sars_rok['confirmed_scaled'], mode = 'lines', name = 'Sars_rok')
trace3 = go.Scatter(x = mres_rok_pre['index'], y = mres_rok_pre['confirmed_scaled'], mode = 'lines', name = 'mres_rok_pre')

data = [trace1, trace2, trace3]
pyo.iplot(data)

- using MinMaxScaler, i can see more detail about the distribution.
- as you can see for COVID 19 and SARS increased rapidly.

### now i will apply time series analytics for corna_rok datset

In [ ]:
corna_rok.head()

- to use ARIMA, i manipulated existing dataset.

In [ ]:
corna_arima = corna_rok[['dates','confirmed']]
corna_arima.head()

In [ ]:
corna_arima.dtypes

- corna_arima is consist of object and int64 data type.
- convert object that is type of 'date' to DatetimeIndex.

In [ ]:
corna_arima['dates'] = pd.DatetimeIndex(corna_arima['dates'])

In [ ]:
corna_arima = corna_arima.rename(columns = {'dates':'ds', 'confirmed':'y'})
corna_arima

- let's convert int64 that is type of 'y' to float. 
- ARIMA model only treats float data type.

In [ ]:
corna_arima['y'] = corna_arima['y'].values.astype(float)

- check the data types

In [ ]:
corna_arima.dtypes

- i will use 'ds' for index

In [ ]:
corna_arima = corna_arima.set_index("ds")

In [ ]:
corna_arima.head()

- in ARIMA model, we should estimate the proper coefficient of pcf and pacf.
- frist, draw some graphs

In [ ]:
import matplotlib.pyplot as plt
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

corna_arima.plot()
plt.show()

In [ ]:
plot_acf(corna_arima)
plot_pacf(corna_arima)
plt.figure(figsize=(20,4))
plt.show()

- it shows us non stationary process, we need to convert non-stationary things to stationary things.
- second, therefore,above the reason, we should apply 'difference' for origin dataset.

In [ ]:
diff_1 = corna_arima.diff(periods=1).iloc[1:]
diff_1.plot()
plot_acf(diff_1)
plot_pacf(diff_1)
plt.show()

- it has a lot of manners choosing pcf and pacf coefficient. 
- i use the simple manner that is choose a point observing autocorrelation graphs of acf and pacf.  

In [ ]:
from statsmodels.tsa.arima_model import ARIMA
from statsmodels.tsa.arima_model import ARIMAResults

In [ ]:
model = ARIMA(corna_arima, order=(2,1,0))
model_fit = model.fit(trend = 'nc', full_output = True, disp=1)
print(model_fit.summary())

In [ ]:
model_fit.plot_predict()
fore = model_fit.forecast(steps = 1)
print(fore)

In [ ]:
# ARIMA의 p,d,q를 변경
model = ARIMA(corna_arima, order=(0,1,1))
model_fit = model.fit(trend = 'nc', full_output = True, disp=1)
print(model_fit.summary())

In [ ]:
model_fit.plot_predict()

# 모델에 돌린걸 예측해서 보여줘라
fore = model_fit.forecast(steps = 1)
print(fore)

In [ ]:
corna_arima.tail()

- 9332

- first model's predict value is '**9336**' and second in '9269'. the predicted date is '03-27'
- the real value of 03-27 is '**9332**'.
- predict value of first model is more accurate that second model.